In [ ]:
## 아래는 실제 트레인 과정 입니다.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import numpy as np
import torch.optim as optim
import pandas as pd
from pandas import DataFrame, Series
import csv
from random import randrange
import pickle

In [2]:
def load_data_real():
    df = pd.read_csv('data.csv', error_bad_lines=False)

    data_mok = df['mok']
    data_hwa = df['hwa']
    data_to = df['to']
    data_gm = df['gm']
    data_su = df['su']
    data_E = df['E']
    data_I = df['I']
    data_S = df['S']
    data_N = df['N']
    data_T = df['T']
    data_F = df['F']
    data_J = df['J']
    data_P = df['P']
    data_job = df['job']
    
    length = len(data_mok)
    
    data = []
    job = []

    for i in range(0,length):
        row = []
        row.append(data_mok[i])
        row.append(data_hwa[i])
        row.append(data_to[i])
        row.append(data_gm[i])
        row.append(data_su[i])
        row.append(data_E[i])
        row.append(data_I[i])
        row.append(data_S[i])
        row.append(data_N[i])
        row.append(data_T[i])
        row.append(data_F[i])
        row.append(data_J[i])
        row.append(data_P[i])
        
        data.append(row)
        label = []
        label.append(data_job[i])
        job.append(label)
    
    return data, job, length

In [3]:
class myDataset(Dataset):
    def __init__(self, features, label):
        self.features = features
        self.label = label
        self.len = len(self.label)
        self.label_list = list(sorted(set(self.label)))
    
    def __getitem__(self, index):
        return self.features[index], self.label[index]
    def __len__(self):
        return self.len
    def get_labels(self):
        return self.label_list
    def get_label(self, id):
        return self.label_list[id]
    def get_label_id(self,label):
        return self.label_list.index(label)

In [8]:
BATCH_SIZE = 16
epochs = 50

data, label, length = load_data_real()

#label indexing
vocab = set()
vocab_label = np.array(label)
vocab.update(vocab_label.flatten())
label_vocab = {word:i for i, word in enumerate(vocab)}
print(label_vocab)
file=open("vocabulary", "wb")
print()
pickle.dump(label_vocab,file)
file.close()
final_label = []
for index, label_ in enumerate(label):
    final_label.append([label_vocab[label_[0]]])
    
data = np.array(data, dtype=np.float32)
final_label = np.array(final_label, dtype=np.float64)
final_label = final_label.flatten()
data = Variable(torch.from_numpy(data))
final_label = Variable(torch.from_numpy(final_label))

train_data = data[:18000]
train_label = final_label[:18000]
test_data = data[18000:length]
test_label = final_label[18000:length]

train_dataset = myDataset(train_data, train_label)
test_dataset = myDataset(test_data, test_label)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE,
                         shuffle=False)

# Training settings
# batch_size = 64

# # MNIST Dataset
# train_dataset = datasets.MNIST(root='./mnist_data/',
#                                train=True,
#                                transform=transforms.ToTensor(),
#                                download=True)

# test_dataset = datasets.MNIST(root='./mnist_data/',
#                               train=False,
#                               transform=transforms.ToTensor())

# # Data Loader (Input Pipeline)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=batch_size,
#                                            shuffle=True)

# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=batch_size,
#                                           shuffle=False)
class myModel(nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(myModel, self).__init__()
        self.l1 = nn.Linear(13,50)
        self.l2 = nn.Linear(50,30)
        self.l3 = nn.Linear(30,15)
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        x = x.view(-1,13)
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(8, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)
# our model
model = myModel()


# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        target = torch.tensor(target,dtype=torch.long)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
            
def my_test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        target = torch.tensor(target,dtype=torch.long)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        target = target.view(pred.size(0),1)
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# Training loop
for epoch in range(epochs):
    train(epoch)
    my_test()
    

{'레스토랑': 0, '사무보조': 1, '의류': 2, '유아': 3, 'PC': 4, '서빙': 5, '생산': 6, '바리스타': 7, '스포츠': 8, '미용': 9, '매장': 10, '회계': 11, '창고': 12, '배달': 13, '주방': 14}


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:149: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [0/18000 (0%)]	Loss: 2.738402
Train Epoch: 0 [1600/18000 (9%)]	Loss: 2.604198
Train Epoch: 0 [3200/18000 (18%)]	Loss: 2.311863
Train Epoch: 0 [4800/18000 (27%)]	Loss: 2.262905
Train Epoch: 0 [6400/18000 (36%)]	Loss: 2.220991
Train Epoch: 0 [8000/18000 (44%)]	Loss: 2.190809
Train Epoch: 0 [9600/18000 (53%)]	Loss: 2.168204
Train Epoch: 0 [11200/18000 (62%)]	Loss: 2.178184
Train Epoch: 0 [12800/18000 (71%)]	Loss: 2.100941
Train Epoch: 0 [14400/18000 (80%)]	Loss: 2.122899
Train Epoch: 0 [16000/18000 (89%)]	Loss: 2.127244
Train Epoch: 0 [17600/18000 (98%)]	Loss: 2.149613


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:160: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.1332, Accuracy: 892/3498 (25%)

Train Epoch: 1 [0/18000 (0%)]	Loss: 2.121195
Train Epoch: 1 [1600/18000 (9%)]	Loss: 2.089332
Train Epoch: 1 [3200/18000 (18%)]	Loss: 2.058708
Train Epoch: 1 [4800/18000 (27%)]	Loss: 2.106593
Train Epoch: 1 [6400/18000 (36%)]	Loss: 2.169744
Train Epoch: 1 [8000/18000 (44%)]	Loss: 2.160427
Train Epoch: 1 [9600/18000 (53%)]	Loss: 2.136431
Train Epoch: 1 [11200/18000 (62%)]	Loss: 2.141215
Train Epoch: 1 [12800/18000 (71%)]	Loss: 2.152282
Train Epoch: 1 [14400/18000 (80%)]	Loss: 2.149994
Train Epoch: 1 [16000/18000 (89%)]	Loss: 2.129666
Train Epoch: 1 [17600/18000 (98%)]	Loss: 2.133674

Test set: Average loss: 0.1326, Accuracy: 893/3498 (25%)

Train Epoch: 2 [0/18000 (0%)]	Loss: 2.133835
Train Epoch: 2 [1600/18000 (9%)]	Loss: 2.170318
Train Epoch: 2 [3200/18000 (18%)]	Loss: 2.141896
Train Epoch: 2 [4800/18000 (27%)]	Loss: 2.121010
Train Epoch: 2 [6400/18000 (36%)]	Loss: 2.168515
Train Epoch: 2 [8000/18000 (44%)]	Loss: 2.135579
Train

Train Epoch: 13 [9600/18000 (53%)]	Loss: 2.117465
Train Epoch: 13 [11200/18000 (62%)]	Loss: 2.044266
Train Epoch: 13 [12800/18000 (71%)]	Loss: 2.086208
Train Epoch: 13 [14400/18000 (80%)]	Loss: 2.207397
Train Epoch: 13 [16000/18000 (89%)]	Loss: 2.103087
Train Epoch: 13 [17600/18000 (98%)]	Loss: 2.081026

Test set: Average loss: 0.1325, Accuracy: 905/3498 (25%)

Train Epoch: 14 [0/18000 (0%)]	Loss: 2.117327
Train Epoch: 14 [1600/18000 (9%)]	Loss: 2.100416
Train Epoch: 14 [3200/18000 (18%)]	Loss: 2.100375
Train Epoch: 14 [4800/18000 (27%)]	Loss: 2.131577
Train Epoch: 14 [6400/18000 (36%)]	Loss: 2.137849
Train Epoch: 14 [8000/18000 (44%)]	Loss: 2.086186
Train Epoch: 14 [9600/18000 (53%)]	Loss: 2.074643
Train Epoch: 14 [11200/18000 (62%)]	Loss: 2.114385
Train Epoch: 14 [12800/18000 (71%)]	Loss: 2.112773
Train Epoch: 14 [14400/18000 (80%)]	Loss: 2.178638
Train Epoch: 14 [16000/18000 (89%)]	Loss: 2.071077
Train Epoch: 14 [17600/18000 (98%)]	Loss: 2.088905

Test set: Average loss: 0.1325, Acc

Train Epoch: 25 [17600/18000 (98%)]	Loss: 2.088913

Test set: Average loss: 0.1325, Accuracy: 890/3498 (25%)

Train Epoch: 26 [0/18000 (0%)]	Loss: 2.129141
Train Epoch: 26 [1600/18000 (9%)]	Loss: 2.168825
Train Epoch: 26 [3200/18000 (18%)]	Loss: 2.088807
Train Epoch: 26 [4800/18000 (27%)]	Loss: 2.100426
Train Epoch: 26 [6400/18000 (36%)]	Loss: 2.154768
Train Epoch: 26 [8000/18000 (44%)]	Loss: 2.126972
Train Epoch: 26 [9600/18000 (53%)]	Loss: 2.132174
Train Epoch: 26 [11200/18000 (62%)]	Loss: 2.157389
Train Epoch: 26 [12800/18000 (71%)]	Loss: 2.171623
Train Epoch: 26 [14400/18000 (80%)]	Loss: 2.128964
Train Epoch: 26 [16000/18000 (89%)]	Loss: 2.165018
Train Epoch: 26 [17600/18000 (98%)]	Loss: 2.086176

Test set: Average loss: 0.1324, Accuracy: 895/3498 (25%)

Train Epoch: 27 [0/18000 (0%)]	Loss: 2.086193
Train Epoch: 27 [1600/18000 (9%)]	Loss: 2.114643
Train Epoch: 27 [3200/18000 (18%)]	Loss: 2.117360
Train Epoch: 27 [4800/18000 (27%)]	Loss: 2.031922
Train Epoch: 27 [6400/18000 (36%)]	L

Train Epoch: 38 [4800/18000 (27%)]	Loss: 2.100390
Train Epoch: 38 [6400/18000 (36%)]	Loss: 2.103105
Train Epoch: 38 [8000/18000 (44%)]	Loss: 2.114685
Train Epoch: 38 [9600/18000 (53%)]	Loss: 2.061326
Train Epoch: 38 [11200/18000 (62%)]	Loss: 2.074620
Train Epoch: 38 [12800/18000 (71%)]	Loss: 2.117341
Train Epoch: 38 [14400/18000 (80%)]	Loss: 2.114655
Train Epoch: 38 [16000/18000 (89%)]	Loss: 2.185678
Train Epoch: 38 [17600/18000 (98%)]	Loss: 2.175861

Test set: Average loss: 0.1330, Accuracy: 891/3498 (25%)

Train Epoch: 39 [0/18000 (0%)]	Loss: 2.098125
Train Epoch: 39 [1600/18000 (9%)]	Loss: 2.239725
Train Epoch: 39 [3200/18000 (18%)]	Loss: 2.171140
Train Epoch: 39 [4800/18000 (27%)]	Loss: 2.118172
Train Epoch: 39 [6400/18000 (36%)]	Loss: 2.103154
Train Epoch: 39 [8000/18000 (44%)]	Loss: 2.148823
Train Epoch: 39 [9600/18000 (53%)]	Loss: 2.137828
Train Epoch: 39 [11200/18000 (62%)]	Loss: 2.060402
Train Epoch: 39 [12800/18000 (71%)]	Loss: 2.100784
Train Epoch: 39 [14400/18000 (80%)]	Los

In [ ]:
##아래는 데이터 만드는 과정입니다.

In [5]:
torch.save(model,'saved_model')

C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:241: UserWarning: Couldn't retrieve source code for container of type myModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [43]:
model2 = torch.load('saved_model')
myinput = [1,2,2,1,0,2,4,2,4,2,4,2,4]
myinput = np.array(myinput,dtype=np.float32)
myinput = Variable(torch.from_numpy(myinput))
output = model2(myinput)
# print(output)
output = output.view(-1)
list = [i[0] for i in sorted(enumerate(output), key=lambda x:x[1], reverse=True)]
# print(output)
pos_list = sorted(output, reverse=True)
# print(pos_list)
# print(list)
# print(label_vocab)
sum_ = sum(pos_list)
# print(sum_)
# print(label_vocab.items())

job_list_ = []
pos_list_ = []

print(pos_list)
for num in pos_list:
    pos_list_.append(num/sum_)

print(pos_list_)
for num in list:    
    a=[name for name, age in label_vocab.items() if age == num]
    job_list_.append(a[0])
    
print(job_list_)

[tensor(1.0000, grad_fn=<SelectBackward>), tensor(1.0000, grad_fn=<SelectBackward>), tensor(1.0000, grad_fn=<SelectBackward>), tensor(1.0000, grad_fn=<SelectBackward>), tensor(1.0000, grad_fn=<SelectBackward>), tensor(0.9999, grad_fn=<SelectBackward>), tensor(3.2098e-06, grad_fn=<SelectBackward>), tensor(1.4244e-06, grad_fn=<SelectBackward>), tensor(1.1012e-06, grad_fn=<SelectBackward>), tensor(3.6805e-07, grad_fn=<SelectBackward>), tensor(1.3170e-07, grad_fn=<SelectBackward>), tensor(3.6566e-08, grad_fn=<SelectBackward>), tensor(9.2664e-09, grad_fn=<SelectBackward>), tensor(4.1110e-09, grad_fn=<SelectBackward>), tensor(8.5897e-10, grad_fn=<SelectBackward>)]
[tensor(0.1667, grad_fn=<DivBackward1>), tensor(0.1667, grad_fn=<DivBackward1>), tensor(0.1667, grad_fn=<DivBackward1>), tensor(0.1667, grad_fn=<DivBackward1>), tensor(0.1667, grad_fn=<DivBackward1>), tensor(0.1667, grad_fn=<DivBackward1>), tensor(5.3498e-07, grad_fn=<DivBackward1>), tensor(2.3740e-07, grad_fn=<DivBackward1>), tens

In [8]:
def load_data():
    df = pd.read_csv('data.csv', error_bad_lines=False)

    data_mok = df['mok']
    data_hwa = df['hwa']
    data_to = df['to']
    data_gm = df['gm']
    data_su = df['su']
    data_E = df['E']
    data_I = df['I']
    data_S = df['S']
    data_N = df['N']
    data_T = df['T']
    data_F = df['F']
    data_J = df['J']
    data_P = df['P']
    
    length = len(data_mok)
    
    data = []

    for i in range(0,length):
        row = []
        row.append(data_mok[i])
        row.append(data_hwa[i])
        row.append(data_to[i])
        row.append(data_gm[i])
        row.append(data_su[i])
        row.append(data_E[i])
        row.append(data_I[i])
        row.append(data_S[i])
        row.append(data_N[i])
        row.append(data_T[i])
        row.append(data_F[i])
        row.append(data_J[i])
        row.append(data_P[i])

        data.append(row)


    print(data[0])
    print(data[1])
    
    com = data[0]
    
    jobs = []
    
    for data_ in data:
        select_list = data_[:5]
        max_ = max(data_[:5])
        list_ = [ i for i, x in enumerate( select_list ) if x == max_ ]
        random_index = randrange(0,len(list_))
        
        last_num = list_[random_index]
        
        if last_num == 0:
            list__ = ['바리스타', '매장', '의류']
            random_index = randrange(0,len(list__))
            job = list__[random_index]
            jobs.append(job)
        elif last_num == 1:
            list__ = ['레스토랑', '사무보조', '미용']
            random_index = randrange(0,len(list__))
            job = list__[random_index]
            jobs.append(job)
        elif last_num == 2:
            list__ = ['생산', '주방', '회계']
            random_index = randrange(0,len(list__))
            job = list__[random_index]
            jobs.append(job)
        elif last_num == 3:
            list__ = ['유아', '스포츠', 'PC']
            random_index = randrange(0,len(list__))
            job = list__[random_index]
            jobs.append(job)
        elif last_num == 4:
            list__ = ['배달', '창고', '서빙']
            random_index = randrange(0,len(list__))
            job = list__[random_index]
            jobs.append(job)
    print(jobs)
    return jobs

In [9]:

jobs = load_data()

data = DataFrame(jobs)
print(data)
data.to_excel('label_2.xlsx', sheet_name='sheet1')
# data.to_csv('label_2.csv', index=False, header=False, encoding='ms949')
# f = open('label.csv', 'w', encoding='euc_kr', newline='')
# wr = csv.writer(f)
# for job in jobs:
#     wr.writerow(job)

[1, 3, 1, 0, 1, 2, 4, 2, 4, 2, 4, 4, 2]
[1, 3, 1, 0, 1, 2, 4, 2, 4, 2, 4, 4, 2]
['미용', '사무보조', '회계', '레스토랑', '사무보조', '생산', '사무보조', '사무보조', '생산', '회계', '미용', '미용', '주방', '생산', '주방', '미용', '레스토랑', '회계', '주방', '사무보조', '레스토랑', '사무보조', '주방', '생산', '주방', '사무보조', '회계', '레스토랑', '사무보조', '주방', '미용', '주방', '회계', '미용', '스포츠', '의류', '바리스타', '주방', '사무보조', '사무보조', '미용', '생산', '스포츠', '주방', '유아', '배달', '주방', '매장', '매장', '생산', '미용', '회계', '회계', '스포츠', '스포츠', '회계', '창고', '바리스타', '생산', '사무보조', '레스토랑', '생산', '사무보조', 'PC', '회계', '스포츠', 'PC', '매장', '바리스타', '미용', '레스토랑', '회계', '매장', '스포츠', '레스토랑', '미용', '생산', 'PC', '의류', '레스토랑', '사무보조', '주방', '생산', '바리스타', '바리스타', '주방', '사무보조', '미용', '사무보조', '사무보조', '생산', '주방', '유아', 'PC', 'PC', '스포츠', '유아', '레스토랑', '레스토랑', 'PC', '스포츠', '스포츠', '스포츠', '스포츠', '유아', '생산', '바리스타', '매장', '레스토랑', '레스토랑', '유아', '회계', '유아', '유아', '스포츠', 'PC', 'PC', 'PC', '유아', 'PC', '생산', '미용', '스포츠', 'PC', 'PC', '유아', 'PC', '미용', '미용', '미용', '사무보조', '레스토랑', '유아', '미용', '사무보조', '사무보조', 'PC', 'PC', '사

          0
0        미용
1      사무보조
2        회계
3      레스토랑
4      사무보조
5        생산
6      사무보조
7      사무보조
8        생산
9        회계
10       미용
11       미용
12       주방
13       생산
14       주방
15       미용
16     레스토랑
17       회계
18       주방
19     사무보조
20     레스토랑
21     사무보조
22       주방
23       생산
24       주방
25     사무보조
26       회계
27     레스토랑
28     사무보조
29       주방
...     ...
21468    창고
21469  레스토랑
21470    주방
21471  레스토랑
21472    PC
21473    회계
21474    주방
21475    창고
21476    회계
21477    서빙
21478    생산
21479    회계
21480    생산
21481    미용
21482    생산
21483    주방
21484    생산
21485    생산
21486    배달
21487    서빙
21488    생산
21489    주방
21490    의류
21491    회계
21492    생산
21493    주방
21494    회계
21495   스포츠
21496   스포츠
21497    주방

[21498 rows x 1 columns]
